# Azure Machine Learning による機械学習プロセス - デプロイ編

## アジェンダ
### A. 学習編
- ワークスペース (Workspace) への接続
- データセット (Datasets) の登録
- 環境 (Environments) の登録
- コンピューティングクラスター (Compute Clusters) の作成
- モデル学習の実行と実験 (Run & Experiments)
- モデル登録 (Models)

### **B. デプロイ編 (本ノートブック)**
- ワークスペース (Workspace) への接続
- 推論環境の作成 (Deployment)
- エンドポイントの利用 (Endpoint)

## 事前設定
- 本ノートブックは Azure Machine Learning の Compute Instance を利用することを想定しています。
- 開発環境は JupyterLab, VSCode, Integrated Notebook など Compute Instance で稼働するものであれば自由に選択いただけます。
- カーネルは `python38-azureml (Python 3.8 AzureML)` を選択ください。

<br>

## ワークスペース (Workspace) への接続
作業環境から Azure Machine Learning Workspace へ接続を行います。

#### Azure Machine Learning Studio
[ml.azureml.com](ml.azurem.com) にアクセスします。Python SDK を中心に作業される場合にも Azure Machine Learning Studio を併用することが多いです。

<img src="../docs/images/azureml-workspace.png" width=500>


#### Python SDK での手順
クライアント環境の Python 環境にインストールした Azure ML Python SDK を用いて Azure Machine Learning Workspace に接続します。

In [1]:
# Compute Instance を利用する場合
from azureml.core import Workspace
ws = Workspace.from_config()

In [2]:
# # その他の任意のクライアント環境を利用する場合
# ws = Workspace.get(
#     name='name',
#     subscription_id='subscription_id',
#     resource_group='resource_group',
# )

<br>

## 推論環境の作成 (Deployment)
下記の情報を利用してモデルをデプロイし、推論環境を作成します。
- 登録済みのモデル (Model)
- 推論環境で稼働する環境 (Environments)
- 推論スクリプト : _score.py_

#### Python SDK での手順

In [3]:
from azureml.core.environment import Environment
from azureml.core import Model
from azureml.core.webservice import LocalWebservice, AciWebservice
from azureml.core.model import InferenceConfig

##### 登録済みのモデル (Models)

In [4]:
model = Model(ws, "lgb-model")

##### 推論環境で稼働する環境 (Environments)

In [5]:
env = Environment.get(ws, "lightgbm-python-env")
#env.inferencing_stack_version = 'latest'

In [6]:
env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220208.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "AZUREML_ENTRY_SCRIPT": "script/score.py",
        "AZUREML_SOURCE_DIRECTORY": "script",
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "lightgbm-python-env",
    "python": {
        "baseCondaEnvironment": null,
        "con

##### 推論スクリプト _score.py_
`script` フォルダに予め作成済みです。

```python

import os
import json
import numpy as np
import pandas as pd
import lightgbm as lgb
from helper import data_preprocess



def init():
    global bst
    model_root = os.getenv("AZUREML_MODEL_DIR")
    # 学習済みモデルを含むフォルダ名の指定
    lgbm_model_folder = "model"
    bst = lgb.Booster(
        model_file=os.path.join(model_root, lgbm_model_folder, "model.lgb")
    )

def run(raw_data):
    categorical_cols = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
    float_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
    columns = bst.feature_name()
    data = np.array(json.loads(raw_data)["data"])
    test_df_original = pd.DataFrame(data=data, columns=columns)
    test_df = data_preprocess(test_df_original, categorical_cols, float_cols)
    # 予測値の生成
    out = bst.predict(test_df)
    return out.tolist()

````

コードや環境 (Environments) の情報を設定します。

In [7]:
inference_config = InferenceConfig(entry_script="score.py", source_directory="script", environment=env)

最初にローカル環境にデプロイをします。

In [8]:
localconfig = LocalWebservice.deploy_configuration(port=8890)
local_service_name = "mylocalmodel"
local_service = Model.deploy(
    workspace=ws,
    name=local_service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=localconfig,
    overwrite=True
)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry ftamlacrnwdem.azurecr.io
Logging into Docker registry ftamlacrnwdem.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM ftamlacrnwdem.azurecr.io/azureml/azureml_e51c371367728f42a881ccad677344ed
 ---> 6f9f959b4c84
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 3689828ce76f
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjljMGY5MWI4LWViMmYtNDg0Yy05NzljLTE1ODQ4YzA5OGE2YiIsInJlc291cmNlR3JvdXBOYW1lIjoiZnRhLW1sIiwiYWNjb3VudE5hbWUiOiJmdGFtbGF6dXJlbWwiLCJ3b3Jrc3BhY2VJZCI6IjRiNzYxNzk0LTQ1OWYtNDc2MS1hZWFjLTVjNzRjZTM3YWZkZCJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in d4cbf3686ccb
 ---> 5948f4fab156
Step 4/5 : RUN mv '/var/azureml-app/tmp7m89nvmj.py' /var/azureml-app/main.py
 ---> Running in a1610d329955
 ---> 8ca95c816f8b
Step 5/5 : CMD ["runsvdir","/var/runit"]
 --

テストデータを入力して予測値を算出します。

In [9]:
import json

# テストデータ
data =  {
            "data": [[
                2,
                "Kvillner, Mr. Johan Henrik Johannesson",
                "male",
                31,
                0,
                0,
                "C.A. 18723",
                10.5,
                "",
                "S"
            ]]
        }

test_sample = json.dumps(data)
test_sample = str.encode(test_sample, encoding='utf8')

prediction = local_service.run(input_data=test_sample)
print(prediction)

Checking container health...
Local webservice is running at http://localhost:8890
[[0.7252904642292589, 0.2747095357707411]]


<br>

次に Azure Container Instance へデプロイをします。

In [10]:
aciconfig = AciWebservice.deploy_configuration(auth_enabled=True)

Azure Container Instance にモデルをデプロイします。<br>
なお、`service_name` は文字から始まる 3 以上 32 小文字・数字・記号 (ダッシュのみ)で記載ください。


In [11]:
aciconfig.validate_configuration()

In [12]:
service_name = "lgb-aci"  # グループで作業している場合は、名前を変更してください
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
    overwrite=True
)

In [13]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-06 14:45:03+00:00 Creating Container Registry if not exists.
2022-04-06 14:45:04+00:00 Registering the environment.
2022-04-06 14:45:04+00:00 Use the existing image.
2022-04-06 14:45:04+00:00 Generating deployment configuration.
2022-04-06 14:45:05+00:00 Submitting deployment to compute.
2022-04-06 14:45:09+00:00 Checking the status of deployment lgb-aci3..
2022-04-06 14:45:29+00:00 Checking the status of inference endpoint lgb-aci3.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
service.get_logs()

'2022-04-06T14:45:18,151287655+00:00 - iot-server/run \n2022-04-06T14:45:18,152994579+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2022-04-06T14:45:18,453755549+00:00 - nginx/run \n2022-04-06T14:45:18,452391231+00:00 - rsyslog/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2022-04-06T14:45:22,054053936+00:00 - iot-server/finish 1 0\n2022-04-06T14:45:22,149499568+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (13)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 38\nSPARK_HOME not set. Skipping PySpark Initialization.\nInitializing logger\n2022-04-06 14:45:38,053 | root | INFO | Starting up app insights client\nlogging socket was found. logging is available.\nlogging socket was found. logging is available.\n2022-04-06 14:45:38,054 | root | INFO | Starting up request id generator\n2022-04-06 14:45:38,054

Azure Machine Learning Studio にて正常に登録されていることを確認します。<br>
<img src="../docs/images/azureml-deployment1.png" width=500><br>



<br>

## エンドポイントの利用 (Endpoint)
推論環境にテストデータをインプットして、デプロイした機械学習モデルから予測値を算出します。

#### Python SDK での手順

In [15]:
import urllib.request
import json
import os
import ssl

# テストデータ
data =  {
            "data": [[
                2,
                "Kvillner, Mr. Johan Henrik Johannesson",
                "male",
                31,
                0,
                0,
                "C.A. 18723",
                10.5,
                "",
                "S"
            ]]
        }
body = str.encode(json.dumps(data), encoding='utf8')

In [16]:
url = service.scoring_uri
key, _ = service.get_keys()
headers = {'Content-Type':'application/json'}
headers["Authorization"] = f"Bearer {key}"
req = urllib.request.Request(url, body, headers)

In [17]:
try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


b'[[0.7252904642292589, 0.2747095357707411]]'
